In [ ]:
import pandas as pd
import re

# suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [ ]:
# column header contains the top row of data; take contents of header and put into a new df

first_row = df_rooms.columns.to_frame().T
first_row

In [ ]:
# add top row to rest of data
# reset index
# rename column headers
# drop the 'comments' column

df_rooms = first_row.append(df_rooms)
df_rooms.reset_index(drop = True, inplace = True)
df_rooms.columns = ['room', 'category', 'comment']
df_rooms.drop('comment', axis = 1, inplace = True)
df_rooms.head()

In [190]:
# extract floor area and capacity into own column

# first define regex: (, then one or more symbols, then )
pattern = re.compile('\(.+\)')

def extract_area(room):
    # use regex to extract floor area from room info
    # remove left bracket, right space, 'SQM' and right bracket
    match = pattern.search(room).group()
    return match.replace('(', '').replace(' SQM)', '')

def extract_capacity(category):
    # set cap to 0 if not applicable (room has no set capacity)
    # otherwise, use regex to extract capacity from category info
    # remove left and right brackets
    
    if ("Not Applicable" in category):
        cap = 0
    else:
        match = pattern.search(category).group()
        cap = match.replace('(', '').replace(')', '')
        
    return cap;
    
# apply extract_area to room column; output into new 'area' column
df_rooms['area'] = df_rooms['room'].apply(extract_area).astype(float)

# apply extract_capacity to category column; output into new 'capacity' column
df_rooms['capacity'] = df_rooms['category'].apply(extract_capacity).astype(int)

print(df_rooms.shape)
df_rooms.head()

(40, 4)


,room,category,area,capacity
0,B00A - R0001 - Home Base - (71.49 SQM) - Teach...,Student Space (33),71.49,33
1,B00A - R0002 - Home Base - (71.4 SQM) - Teachi...,Student Space (33),71.40,33
2,B00A - R0003 - Movement - (79.87 SQM),Not Applicable,79.87,0
3,B00A - R0004 - Interview - (12.59 SQM),Office (3),12.59,3
4,B00A - R0005 - Principal - (18.62 SQM),Office (4),18.62,4


In [ ]:
# extract cat A into own df

def is_catA(category):
    pattern = re.compile("Student Space")
    return pattern.search(category) != None

df_catA = df_rooms[df_rooms['category'].apply(is_catA)]

print(df_catA.shape)
df_catA.head()

In [ ]:
# use formula to guess cat A capacities
# compare guess with actual capacity

df_catA['capacity_guess'] = df_catA['area'].apply(lambda x: int(x*0.925/2))
df_catA['delta'] = df_catA['capacity'] - df_catA['capacity_guess']

print(df_catA.head())

In [ ]:
df_catA['delta'].value_counts()